In [1]:
import cropvalidation 
import os
import numpy as np
import pandas as pd

### TOGO 


Load data

load in togo data as cropmasks

In [2]:
togo_data = []
dir = 'data/TogoBinaryMaps'
for tif in os.listdir(dir):
    f = os.path.join(dir, tif)
    if f == "data/TogoBinaryMaps\AgreementMap_Togo_Binary.tif":
        continue
    togo_data.append(cropvalidation.CropMask(f))

togo_labels = [togo_data[x].getName().split("\\")[-1][:-11] for x in np.arange(len(togo_data))]

TOGO_BINARY = dict(zip(togo_labels, togo_data))


Matched Sets

In [3]:
TOGO_VALIDATION = cropvalidation.ValidationSet(path = 'data/HarvestData/togo.csv', projection='epsg:4326')
TOGO_VALIDATION.validation = TOGO_VALIDATION.validation[TOGO_VALIDATION.validation['subset'] == 'testing']
TOGO_VALIDATION.validation = TOGO_VALIDATION.validation[['lat', 'lon', 'crop_probability']]

TOGO_VALIDATION.validation.crop_probability = TOGO_VALIDATION.validation.crop_probability.astype(float)

TOGO_VALIDATION.filter()
TOGO_VALIDATION.reproject_validation(outProjection='epsg:32631')


In [4]:
TOGO_VALIDATION.getData()

,lat,lon,crop_probability
77,170560.155817,1.188884e+06,1.0
116,172692.923816,1.193288e+06,0.0
140,177454.027895,1.221734e+06,1.0
145,181613.837353,1.171039e+06,1.0
218,194962.643680,1.214083e+06,1.0
...,...,...,...
1522,345720.588089,8.986492e+05,0.0
1523,345633.778487,8.548428e+05,1.0
1531,346580.577840,9.969522e+05,0.0
1544,346361.833763,7.019713e+05,1.0


In [5]:
TOGO_MATCHED = []
for binarymap in np.arange(len(togo_data)):
    TOGO_MATCHED.append(cropvalidation.MatchedSet(togo_data[binarymap], TOGO_VALIDATION))

In [6]:
TOGO_MATCHED[0].data

,lat,lon,crop_probability,cropmask
0,170560.155817,1.188884e+06,1.0,1.0
1,172692.923816,1.193288e+06,0.0,1.0
2,177454.027895,1.221734e+06,1.0,1.0
3,181613.837353,1.171039e+06,1.0,1.0
4,194962.643680,1.214083e+06,1.0,1.0
...,...,...,...,...
180,345720.588089,8.986492e+05,0.0,1.0
181,345633.778487,8.548428e+05,1.0,1.0
182,346580.577840,9.969522e+05,0.0,0.0
183,346361.833763,7.019713e+05,1.0,1.0


In [7]:
TOGO_F1_SCORES = []
for binarymap in np.arange(len(TOGO_MATCHED)):
    #TOGO_MATCHED[binarymap].remove_filler()
    TOGO_F1_SCORES.append(TOGO_MATCHED[binarymap].get_f1())

TOGO_F1_SCORES = pd.DataFrame(data=zip(TOGO_F1_SCORES, [x.split('/')[-1] for x in togo_labels]))
TOGO_F1_SCORES

,0,1
0,0.471910,ASAP_Togo
1,0.592593,CombinedMap_Togo
2,0.205882,Copernicus_Togo
3,0.634146,DigitalEarth_Togo
4,0.707317,ESA_Togo
5,0.107143,ESRI_Togo
6,0.615385,GFSAD_Togo
7,0.779661,Harvest_Togo


KENYA DATA

In [1]:
import cropvalidation 
import os
import numpy as np
import pandas as pd

In [2]:
kenya_data = []
dir = 'data/KenyaBinaryMaps'
for tif in os.listdir(dir):
    f = os.path.join(dir, tif)
    if f == "data/KenyaBinaryMaps/Agreement_Kenya_Binary.tif":
        continue
    kenya_data.append(cropvalidation.CropMask(f))

kenya_labels = [kenya_data[x].getName().split("\\")[-1][:-11] for x in np.arange(len(kenya_data))]

KENYA_BINARY = dict(zip(kenya_labels, kenya_data))
kenya_data[0].getProjection()


CRS.from_epsg(32736)

In [3]:
KENYA_VALIDATION = cropvalidation.ValidationSet(path = 'data/HarvestData/kenya.csv', projection='epsg:4326')
KENYA_VALIDATION.validation = KENYA_VALIDATION.validation[KENYA_VALIDATION.validation['subset'] == 'testing']
KENYA_VALIDATION.validation = KENYA_VALIDATION.validation[['lat', 'lon', 'crop_probability']]

KENYA_VALIDATION.validation.crop_probability = KENYA_VALIDATION.validation.crop_probability.astype(float)

KENYA_VALIDATION.filter()
KENYA_VALIDATION.reproject_validation(outProjection='epsg:32736')


In [4]:
KENYA_MATCHED = []
for binarymap in np.arange(len(kenya_data)):
    KENYA_MATCHED.append(cropvalidation.MatchedSet(kenya_data[binarymap], KENYA_VALIDATION))

In [7]:
KENYA_F1_SCORES = []
for binarymap in np.arange(len(KENYA_MATCHED)):
    KENYA_MATCHED[binarymap].remove_filler()
    KENYA_F1_SCORES.append(KENYA_MATCHED[binarymap].get_f1())

KENYA_F1_SCORES = pd.DataFrame(data=zip(KENYA_F1_SCORES, [x.split('/')[-1] for x in kenya_labels]))
KENYA_F1_SCORES

,0,1
0,0.068966,Agreement_Kenya
1,0.963739,Asap_Kenya
2,0.965633,CombinedMap_Kenya
3,0.190193,Copernicus_Kenya
4,0.932575,DE_Kenya
5,0.305132,EsaWorldcover_Kenya
6,0.707724,ESRI_Kenya
7,0.973451,GFSAD_Kenya
8,0.967309,Harvest_Kenya


Togo classification report

In [ ]:
TOGO_REPORT = pd.DataFrame(columns=['F1 Score', 'Total Accuracy', 'Crop Precision', 'Non-Crop Precision', 'Crop Recall', 'Non-Crop Recall'], index = [x.split('/')[-1] for x in togo_labels])

for binarymap in np.arange(len(TOGO_MATCHED)):
    TOGO_MATCHED[binarymap].remove_filler()
    TOGO_REPORT.iloc[binarymap] = TOGO_MATCHED[binarymap].report()

In [ ]:
TOGO_REPORT.sort_values(by = 'Total Accuracy', ascending=True)

,F1 Score,Total Accuracy,Crop Precision,Non-Crop Precision,Crop Recall,Non-Crop Recall
Copernicus_Togo,0.205882,0.708108,0.411765,0.738095,0.925373,0.137255
ESRI_Togo,0.107143,0.72973,0.6,0.733333,0.985075,0.058824
ASAP_Togo,0.47191,0.745946,0.552632,0.795918,0.873134,0.411765
GFSAD_Togo,0.615385,0.783784,0.603774,0.856061,0.843284,0.627451
CombinedMap_Togo,0.592593,0.821622,0.8,0.825806,0.955224,0.470588
DigitalEarth_Togo,0.634146,0.837838,0.83871,0.837662,0.962687,0.509804
Harvest_Togo,0.779661,0.859459,0.686567,0.957627,0.843284,0.901961
ESA_Togo,0.707317,0.87027,0.935484,0.857143,0.985075,0.568627


Kenya classification report

In [ ]:
kenya_labels = [kenya_data[x].getName().split("\\")[-1][:-11] for x in np.arange(len(kenya_data))]
KENYA_REPORT = pd.DataFrame(columns=['F1 Score', 'Total Accuracy', 'Crop Precision', 'Non-Crop Precision', 'Crop Recall', 'Non-Crop Recall'], index = [x.split('/')[-1] for x in kenya_labels])

for binarymap in np.arange(len(KENYA_MATCHED)):
    KENYA_MATCHED[binarymap].remove_filler()
    KENYA_REPORT.iloc[binarymap] = KENYA_MATCHED[binarymap].report()


In [ ]:
KENYA_REPORT.sort_values(by = 'Total Accuracy', ascending=False)

,F1 Score,Total Accuracy,Crop Precision,Non-Crop Precision,Crop Recall,Non-Crop Recall
GFSAD_Kenya,0.973451,0.959108,0.951258,0.988304,0.845,0.996705
Harvest_Kenya,0.967309,0.951673,0.984642,0.864253,0.955,0.950577
CombinedMap_Kenya,0.965633,0.949195,0.982935,0.859729,0.95,0.948929
Asap_Kenya,0.963739,0.944238,0.943218,0.947977,0.82,0.985173
DE_Kenya,0.932575,0.89715,0.919872,0.819672,0.75,0.945634
Agreement_Kenya,0.068966,0.834356,0.035714,1.0,0.833333,1.0
ESRI_Kenya,0.707724,0.653036,0.965812,0.412281,0.94,0.558484
EsaWorldcover_Kenya,0.305132,0.379182,0.964912,0.282828,0.98,0.181219
Copernicus_Kenya,0.190193,0.324659,0.969697,0.267206,0.99,0.105437


Scores to add
- user?
- cropland evaluation & non cropland evaluation